<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/Statistika_v_Python/Kejs_stadi/DZ_Kondratev_STPY-48_stat_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции "Статистика. Практика"

## Задание 1

Вернемся к [набору данных о видеоиграх](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/vgsales.csv).

Ответьте на следующие вопросы:

1) Как критики относятся к спортивным играм?  
2) Критикам нравятся больше игры на PC или на PS4?  
3) Критикам больше нравятся стрелялки или стратегии?  

Для каждого вопроса:
- сформулируйте нулевую и альтернативную гипотезы;
- выберите пороговый уровень статистической значимости;
- опишите полученные результаты статистического теста.

## Задание 2

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные [здесь](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/spam.csv)) по признаку спама. Для этого:

1) Привидите весь текст к нижнему регистру;  
2) Удалите мусорные символы;  
3) Удалите стоп-слова;  
4) Привидите все слова к нормальной форме;  
5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:  

```
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.Message)
names = tfidf.get_feature_names_out()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
```

Можете поэкспериментировать с параметрами [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html);  
6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите `random_state=42`. Используйте [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html);  
7) Постройте модель [логистической регрессии](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), укажите `random_state=42`, оцените ее точность на тестовых данных;  
8) Опишите результаты при помощи [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html?highlight=confusion_matrix#sklearn.metrics.confusion_matrix);  
9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

### импорт библиотек

In [279]:
# импорт библиотек
import glob
import pandas as pd
import re

### Загружаем данные

In [280]:
data = pd.read_csv('https://raw.githubusercontent.com/stefkong1982/netology.ru/Master/Statistika_v_Python/Kejs_stadi/spam.csv')
# Загружаем данные
data

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


### Приведение текста к нижнему регистру

In [281]:
# Приведение текста к нижнему регистру
data['Message'] = data['Message'].str.lower()
data

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,this is the 2nd time we have tried 2 contact u...
5568,ham,will ü b going to esplanade fr home?
5569,ham,"pity, * was in mood for that. so...any other s..."
5570,ham,the guy did some bitching but i acted like i'd...


### Удаление мусорных символов

In [282]:
# Удаление мусорных символов
for i in range(len(data['Message'])):
    data['Message'][i] = re.sub(r'[^a-zA-Zа-яА-Я0-9\s]', '', data['Message'][i])
data

,Category,Message
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...
...,...,...
5567,spam,this is the 2nd time we have tried 2 contact u...
5568,ham,will b going to esplanade fr home
5569,ham,pity was in mood for that soany other suggest...
5570,ham,the guy did some bitching but i acted like id ...


### Удаление стоп-слов

In [283]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

filtered_text = []

for text in data['Message']:
    words = text.split()
    filtered_words = []
    for word in words:
        if word not in stop_words:
            filtered_words.append(word)
    filtered_text.append(' '.join(filtered_words))

data['Message'] = filtered_text
data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Category,Message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah dont think goes usf lives around though
...,...,...
5567,spam,2nd time tried 2 contact u u 750 pound prize 2...
5568,ham,b going esplanade fr home
5569,ham,pity mood soany suggestions
5570,ham,guy bitching acted like id interested buying s...


### Приведение слов к нормальной форме

In [284]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
lemmatized_text = []

for text in data['Message']:
    words = text.split()
    lemmatized_words = []
    for word in words:
        lemmatized_word = lemmatizer.lemmatize(word)
        lemmatized_words.append(lemmatized_word)
    lemmatized_text.append(' '.join(lemmatized_words))

data['Message'] = lemmatized_text
data

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Category,Message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah dont think go usf life around though
...,...,...
5567,spam,2nd time tried 2 contact u u 750 pound prize 2...
5568,ham,b going esplanade fr home
5569,ham,pity mood soany suggestion
5570,ham,guy bitching acted like id interested buying s...


### Преобразование сообщений в вектора TF-IDF

In [285]:
# Преобразование сообщений в вектора TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data.Message)
names = tfidf.get_feature_names_out()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
tfidf_matrix

,008704050406,0089my,0121,01223585236,01223585334,0125698789,02,020603,0207,02070836089,...,zebra,zed,zero,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Разделение данных на тестовую и тренировочную выборки

In [286]:
# Разделение данных на тестовую и тренировочную выборки
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, data.Category, test_size=0.3, random_state=42)

In [287]:
y_train

708     ham
4338    ham
5029    ham
4921    ham
2592    ham
       ... 
3772    ham
5191    ham
5226    ham
5390    ham
860     ham
Name: Category, Length: 3900, dtype: object

In [288]:
y_test

3245     ham
944      ham
1044     ham
2484     ham
812      ham
        ... 
2505     ham
2525    spam
4975     ham
650     spam
4463     ham
Name: Category, Length: 1672, dtype: object

### Построение модели логистической регрессии

In [289]:
# Построение модели логистической регрессии
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

### Оценка точности модели на тестовых данных

In [290]:
# Оценка точности модели на тестовых данных
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9539473684210527


Модель достигла точности 95.4% на тестовых данных, что говорит о её способности довольно точно предсказывать, является ли сообщение спамом или нет.

### Создание confusion matrix

In [291]:
# Создание confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:")
print(cm)

Confusion matrix:
[[1444    4]
 [  73  151]]


Верно классифицировано:
- 1444 сообщений, которые являются неспамом и были правильно классифицированы как неспам.
- 151 сообщений, которые являются спамом и были правильно классифицированы как спам.

Неверно классифицировано:
- 4 сообщения, которые являются неспамом, но были ошибочно классифицированы как спам.
- 73 сообщения, которые являются спамом, но были ошибочно классифицированы как неспам.

Вывод:

Модель показывает высокую точность и может быть использована для классификации сообщений как спам или неспам с высокой достоверностью. Однако, она делает некоторое количество ошибок, классифицируя некоторые неспамовые сообщения как спам и наоборот.

### Создание датафрейма с неправильно классифицированными сообщениями

In [292]:
misclassified_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
misclassified_df = misclassified_df[misclassified_df['Actual'] != misclassified_df['Predicted']]
misclassified_df['Message'] = data.loc[misclassified_df.index, 'Message']
misclassified_df = misclassified_df.reset_index(drop=True)
print("Misclassified messages:")
misclassified_df

Misclassified messages:


,Actual,Predicted,Message
0,ham,spam,hey free call
1,spam,ham,reminder downloaded content already paid goto ...
2,spam,ham,guess somebody know secretly fancy wanna find ...
3,spam,ham,oh god ive found number im glad text back xaft...
4,spam,ham,next amazing xxx picsfree1 video sent enjoy on...
...,...,...,...
72,spam,ham,unique enough find 30th august wwwareyouunique...
73,spam,ham,freemsg hi baby wow got new cam moby wanna c h...
74,spam,ham,important customer service announcement premier
75,ham,spam,free nowcan call


Этот код создает датафрейм `misclassified_df` содержащий информацию об ошибках классификации модели. В нем есть следующие столбцы:
- `Actual` - фактическая категория сообщения (из тестовой выборки)
- `Predicted` - предсказанная моделью категория сообщения
- `Message` - текст сообщения

Сначала создается пустой датафрейм, где столбцы `Actual` и `Predicted` заполняются фактическими и предсказанными категориями соответственно. Затем в этом датафрейме оставляются только строки, где фактическая и предсказанная категории не совпадают.

Далее в датафрейме создается столбец `Message`, в котором содержится текст ошибочно классифицированного сообщения. Для этого используется функция `loc`, которая возвращает значения из столбца `Message` в исходных данных (`data`) по индексам строк, которые остались в `misclassified_df`.

Наконец, индексы в `misclassified_df` переиндексируются с помощью `reset_index`(`drop=True`), чтобы они шли последовательно от 0 до N-1, где N - количество ошибочно классифицированных сообщений.

Выводится датафрейм `misclassified_df` с текстом ошибочно классифицированных сообщений.

в количестве 77 строк (4 +73):
```
Confusion matrix:
[[1444    4]
 [  73  151]]
```